In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
%matplotlib inline

In [32]:
prices = pd.DataFrame()

tickers = ['HSBA.UK', 'BARC.UK', 'LLOY.UK', 'RBS.UK', 'BP.UK', 'RIO.UK', 'AAL.UK']

df_list = []

for ticker in tickers:
    df = pd.read_csv(ticker + '.csv')
    df.Date = pd.to_datetime(df.Date)
    df = df.set_index('Date')
    df_list.append(df)

In [33]:
RDSA = pd.read_csv('RDSA.UK.csv', sep = ';')

RDSA.Close = RDSA.Close.apply(lambda x: x.replace(',','.'))
RDSA.Close = RDSA.Close.astype('float64')
RDSA.Date = pd.to_datetime(RDSA.Date, format='%d/%m/%Y')

RDSA = RDSA.set_index('Date')

df_list.append(RDSA)
tickers.append('RDSA.UK')

In [34]:
prices = pd.concat(df_list, axis=1, join='outer')
prices.columns = tickers
prices = prices.sort_index()

In [35]:
prices.head()

,HSBA.UK,BARC.UK,LLOY.UK,RBS.UK,BP.UK,RIO.UK,AAL.UK,RDSA.UK
Date,,,,,,,,
2009-01-07,427.80,144.11,73.682,486.03,409.64,1148.67,1521.30,18.05
2009-01-08,425.98,150.20,76.360,501.90,416.09,1096.69,1431.35,18.41
2009-01-09,418.16,151.91,78.201,526.70,405.93,1033.30,1314.05,17.88
2009-01-12,424.96,156.68,83.674,545.54,398.11,1018.74,1279.65,17.73
2009-01-13,425.79,140.79,79.094,506.86,403.18,985.76,1260.24,18.02


In [36]:
prices.index

DatetimeIndex(['2009-01-07', '2009-01-08', '2009-01-09', '2009-01-12',
               '2009-01-13', '2009-01-14', '2009-01-15', '2009-01-16',
               '2009-01-19', '2009-01-20',
               ...
               '2011-12-21', '2011-12-22', '2011-12-23', '2011-12-28',
               '2011-12-29', '2011-12-30', '2012-01-03', '2012-01-04',
               '2012-01-05', '2012-01-06'],
              dtype='datetime64[ns]', name='Date', length=758, freq=None)

In [37]:
prices.dtypes

HSBA.UK    float64
BARC.UK    float64
LLOY.UK    float64
RBS.UK     float64
BP.UK      float64
RIO.UK     float64
AAL.UK     float64
RDSA.UK    float64
dtype: object

Датафрейм собран, все в нужном формате. Идем дальше.

## Пропуски

In [38]:
prices.isna().sum()

HSBA.UK    8
BARC.UK    1
LLOY.UK    1
RBS.UK     0
BP.UK      9
RIO.UK     5
AAL.UK     0
RDSA.UK    0
dtype: int64

Заменим пропуски интерполяцией.

In [39]:
prices = prices.interpolate(method = 'time', axis = 0)

In [40]:
prices.isna().sum()

HSBA.UK    0
BARC.UK    0
LLOY.UK    0
RBS.UK     0
BP.UK      0
RIO.UK     0
AAL.UK     0
RDSA.UK    0
dtype: int64

Поставим колонки по секторам. Сначала - банковский, затем - нефтегаз, в конце - металлургия.

In [41]:
new_order = ['HSBA.UK', 'BARC.UK', 'LLOY.UK', 'RBS.UK', 'BP.UK', 'RDSA.UK', 'RIO.UK', 'AAL.UK']

prices = prices[new_order]
prices.head()

,HSBA.UK,BARC.UK,LLOY.UK,RBS.UK,BP.UK,RDSA.UK,RIO.UK,AAL.UK
Date,,,,,,,,
2009-01-07,427.80,144.11,73.682,486.03,409.64,18.05,1148.67,1521.30
2009-01-08,425.98,150.20,76.360,501.90,416.09,18.41,1096.69,1431.35
2009-01-09,418.16,151.91,78.201,526.70,405.93,17.88,1033.30,1314.05
2009-01-12,424.96,156.68,83.674,545.54,398.11,17.73,1018.74,1279.65
2009-01-13,425.79,140.79,79.094,506.86,403.18,18.02,985.76,1260.24


In [42]:
prices.to_csv('stock_prices.csv')

## Получим доходности

Получим однодневные.

In [14]:
day1_returns = prices.pct_change()

In [43]:
day1_returns.head()

,HSBA.UK,BARC.UK,LLOY.UK,RBS.UK,BP.UK,RDSA.UK,RIO.UK,AAL.UK
Date,,,,,,,,
2009-01-08,-0.004254,0.042259,0.036345,0.032652,0.015746,0.019945,-0.045252,-0.059127
2009-01-09,-0.018358,0.011385,0.024109,0.049412,-0.024418,-0.028789,-0.057801,-0.081951
2009-01-12,0.016262,0.031400,0.069986,0.035770,-0.019264,-0.008389,-0.014091,-0.026179
2009-01-13,0.001953,-0.101417,-0.054736,-0.070902,0.012735,0.016356,-0.032373,-0.015168
2009-01-14,-0.080039,-0.143405,-0.118770,-0.183956,-0.051887,-0.051054,-0.112553,-0.095164


In [44]:
day1_returns = day1_returns[1:]

In [17]:
day1_returns.head()

,HSBA.UK,BARC.UK,LLOY.UK,RBS.UK,BP.UK,RDSA.UK,RIO.UK,AAL.UK
Date,,,,,,,,
2009-01-08,-0.004254,0.042259,0.036345,0.032652,0.015746,0.019945,-0.045252,-0.059127
2009-01-09,-0.018358,0.011385,0.024109,0.049412,-0.024418,-0.028789,-0.057801,-0.081951
2009-01-12,0.016262,0.031400,0.069986,0.035770,-0.019264,-0.008389,-0.014091,-0.026179
2009-01-13,0.001953,-0.101417,-0.054736,-0.070902,0.012735,0.016356,-0.032373,-0.015168
2009-01-14,-0.080039,-0.143405,-0.118770,-0.183956,-0.051887,-0.051054,-0.112553,-0.095164


In [18]:
day1_returns.to_csv('1_day_returns.csv')

Получим 10-дневные.

In [19]:
prices.head()

,HSBA.UK,BARC.UK,LLOY.UK,RBS.UK,BP.UK,RDSA.UK,RIO.UK,AAL.UK
Date,,,,,,,,
2009-01-07,427.80,144.11,73.682,486.03,409.64,18.05,1148.67,1521.30
2009-01-08,425.98,150.20,76.360,501.90,416.09,18.41,1096.69,1431.35
2009-01-09,418.16,151.91,78.201,526.70,405.93,17.88,1033.30,1314.05
2009-01-12,424.96,156.68,83.674,545.54,398.11,17.73,1018.74,1279.65
2009-01-13,425.79,140.79,79.094,506.86,403.18,18.02,985.76,1260.24


In [24]:
day10_returns = prices.pct_change(periods=9)

In [25]:
day10_returns[:20]

,HSBA.UK,BARC.UK,LLOY.UK,RBS.UK,BP.UK,RDSA.UK,RIO.UK,AAL.UK
Date,,,,,,,,
2009-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
day10_returns = day10_returns[9:]

In [29]:
day10_returns.head()

,HSBA.UK,BARC.UK,LLOY.UK,RBS.UK,BP.UK,RDSA.UK,RIO.UK,AAL.UK
Date,,,,,,,,
2009-01-20,-0.245792,-0.570689,-0.638392,-0.789787,-0.049165,-0.039889,-0.185980,-0.242904
2009-01-21,-0.194892,-0.626511,-0.648756,-0.752959,-0.086976,-0.089082,-0.127155,-0.189779
2009-01-22,-0.161517,-0.669258,-0.626603,-0.770249,-0.056389,-0.062640,-0.057660,-0.148328
2009-01-23,-0.192959,-0.722670,-0.649604,-0.779998,-0.042250,-0.033841,-0.052290,-0.124745
2009-01-26,-0.154677,-0.465324,-0.509773,-0.716253,-0.014559,-0.006659,0.064955,-0.046174


In [30]:
day10_returns.to_csv('10_days_returns.csv')

Получим 10-дневные.

In [48]:
log_returns = np.log(prices)/np.log(prices.shift(1))

In [49]:
log_returns.head()

,HSBA.UK,BARC.UK,LLOY.UK,RBS.UK,BP.UK,RDSA.UK,RIO.UK,AAL.UK
Date,,,,,,,,
2009-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-08,0.999296,1.008327,1.008303,1.005194,1.002597,1.006826,0.993428,0.991682
2009-01-09,0.996940,1.002259,1.005495,1.007756,0.995901,0.989972,0.991494,0.988233
2009-01-12,1.002673,1.006155,1.015518,1.005608,0.996761,0.997079,0.997955,0.996306
2009-01-13,1.000322,0.978842,0.987284,0.988330,1.002114,1.005643,0.995249,0.997864


In [50]:
log_returns = log_returns[1:]

In [51]:
log_returns.to_csv('log_returns.csv')